<a href="https://colab.research.google.com/github/frfrsimp/SINAYA-A-Fuzzy-Neural-Network/blob/main/SINAYA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow numpy pandas

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from tensorflow.keras.metrics import Accuracy, MeanSquaredError
from sklearn.preprocessing import MinMaxScaler

# Load the training data from a CSV file
data = pd.read_csv("/content/Training Dataset v2.csv")

# Normalize the input and output data
scaler = MinMaxScaler()
X = scaler.fit_transform(data[["pH", "do", "temp"]])
y = scaler.fit_transform(data[["ammonia"]])

# Define the input layers
input_pH = tf.keras.layers.Input(shape=(1,))
input_do = tf.keras.layers.Input(shape=(1,))
input_temp = tf.keras.layers.Input(shape=(1,))

# Define the fuzzy membership functions
pH = tf.keras.layers.Lambda(lambda x: tf.clip_by_value(tf.pow(tf.subtract(tf.divide(x, 6.96), 0.765), 3), 0, 1))(input_pH)
do = tf.keras.layers.Lambda(lambda x: tf.clip_by_value(tf.pow(tf.divide(tf.subtract(x, 0.26), 7.3), 2), 0, 1))(input_do)
temp = tf.keras.layers.Lambda(lambda x: tf.clip_by_value(tf.divide(tf.subtract(x, 29.06), 3.27), 0, 1))(input_temp)

# Concatenate the fuzzy membership functions
concat = tf.keras.layers.Concatenate()([pH, do, temp])

# Define the fuzzy rule base
rule1 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], x[1]))([pH, do])
rule2 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], x[1]))([do, temp])
rule3 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], x[1]))([pH, temp])
rule4 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], tf.minimum(x[1], x[2])))([pH, do, temp])
rule5 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], x[1]))([pH, tf.subtract(1.0, do)])
rule6 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], tf.subtract(1.0, tf.clip_by_value(x[1], 0, 1))))([tf.subtract(1.0, pH), temp])
rule7 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], tf.subtract(1.0, x[1])))([do, tf.subtract(1.0, pH)])
rule8 = tf.keras.layers.Lambda(lambda x: tf.minimum(x[0], tf.subtract(1.0, x[1])))([temp, pH])

# Concatenate the rules

# Concatenate the rules
rules = tf.keras.layers.Concatenate()([rule1, rule2, rule3, rule4])

# Define the output layer
output = tf.keras.layers.Dense(1, activation='linear')(rules)

# Define the neural network model
model = tf.keras.models.Model(inputs=[input_pH, input_do, input_temp], outputs=output)

# Train the model
model = Sequential()
model.add(Dense(10, input_shape=(3,), activation='relu'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
optimizer = optimizers.Nadam(learning_rate=0.001, beta_1=0.6, beta_2=0.665, epsilon=7e-08)
model.fit(x=X, y=y, epochs=3000, batch_size=64)
# Save the model
model.save('SINAYA_FFNN_FINAL4.h5')

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

# Evaluate the model on the training set
mse, accuracy = model.evaluate(X, y)

y_pred = model.predict(X)
r2 = r2_score(y, y_pred)
mae = mean_absolute_error(y, y_pred)
print("R^2:", r2)
print("MAE:", mae)
print("MSE on training set:", mse)
print('Accuracy on test set:', accuracy)

# Load the validation data from a CSV file
data2 = pd.read_csv("/content/Validation Dataset v2.csv")

# Normalize the input
X = scaler.fit_transform(data[["pH", "do", "temp"]])
x1 = data2[["pH", "do", "temp"]]
input_data = scaler.transform(x1)

# Feed the input data to the model and get the output
output = model.predict(input_data)

# Do something with the output
output = output * (7.44 - 0.16) + 0.16
print(output)